In [ ]:
from dask.distributed import Client
from pandas import DataFrame

import fastparquet as fp
import numpy as np
import pandas as pd

from datashader.bundling import bundle, nop_bundle
from datashader.colors import inferno, viridis
from datashader.utils import export_image

import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
def render_points(df, width=4000, height=4000, cmap=None, bgcolor=None):
    cvs = ds.Canvas(plot_width=width, plot_height=height, x_range=(0, 1), y_range=(0, 1))
    agg = cvs.points(df, 'x', 'y',  ds.count())
    img = tf.shade(agg, cmap=cmap) if cmap else tf.shade(agg)
    img = tf.spread(img)
    return tf.set_background(img, bgcolor) if bgcolor else img

In [ ]:
def render_lines(df, width=4000, height=4000, cmap=None, bgcolor=None):
    cvs = ds.Canvas(plot_width=width, plot_height=height, x_range=(0, 1), y_range=(0, 1))
    agg = cvs.line(df, 'x', 'y',  ds.count())
    img = tf.shade(agg, cmap=cmap) if cmap else tf.shade(agg)
    return tf.set_background(img, bgcolor) if bgcolor else img

In [ ]:
client = Client()

### Edge bundling with graph read from Parquet

In [ ]:
nodes_df = fp.ParquetFile('nodes.snappy.parq').to_pandas(index='id')
edges_df = fp.ParquetFile('edges.snappy.parq').to_pandas(index='id')

In [ ]:
nodes_df.head()

In [ ]:
edges_df.head()

#### Graph with only nodes

In [ ]:
render_points(nodes_df, width=2000, height=2000)

#### Graph without edge bundling

In [ ]:
nop_df = nop_bundle(nodes_df, edges_df)
render_lines(nop_df, width=2000, height=2000)

#### Graph with edge bundling

In [ ]:
%time df = bundle(nodes_df, edges_df, 0.05, 0.7)

In [ ]:
render_lines(df, width=2000, height=2000, cmap='blue')

In [ ]:
render_lines(df, width=2000, height=2000, cmap='red')

In [ ]:
render_lines(df, width=2000, height=2000, cmap=inferno, bgcolor='black')

In [ ]:
render_lines(df, width=2000, height=2000, cmap=viridis, bgcolor='black')

### Edge bundling with random graph

In [ ]:
def generate_nodes(n):
    return pd.DataFrame(np.random.randn(n, 2), columns=['x', 'y'])

In [ ]:
def generate_edges(n, nodes):
    return pd.DataFrame(np.random.randint(len(nodes), size=(n, 2)), columns=['source', 'target'])

In [ ]:
def generate_random_graph(nodes, edges):
    ndf = generate_nodes(nodes)
    edf = generate_edges(edges, ndf)
    return ndf, edf

In [ ]:
nodes_df, edges_df = generate_random_graph(10000, 50000)

In [ ]:
%time df = bundle(nodes_df, edges_df, 0.05, 0.7)

In [ ]:
render_lines(df, width=2000, height=2000, cmap=viridis, bgcolor='black')

### Edge bundling with star graph and circular layout

In [ ]:
import networkx as nx

In [ ]:
graph = nx.star_graph(100000)

In [ ]:
layout = nx.circular_layout(graph)

In [ ]:
data = []
for node in graph.nodes_iter():
    x, y = layout[node]
    data.append([node, x, y])
nodes_df = pd.DataFrame(data, columns=['id', 'x', 'y'])
nodes_df.set_index('id', inplace=True)

In [ ]:
edges_df = pd.DataFrame(graph.edges_iter(), columns=['source', 'target'])

In [ ]:
%time df = bundle(nodes_df, edges_df, 0.05, 0.7)

In [ ]:
render_lines(df, width=2000, height=2000, cmap=inferno, bgcolor='black')

#### Decrease decay for edge bundling

In [ ]:
%time df = bundle(nodes_df, edges_df, initial_bandwidth=0.05, decay=0.4) # Decay from 0.7 to 0.4

In [ ]:
render_lines(df, width=2000, height=2000, cmap=inferno, bgcolor='black')

#### Increase decay for edge bundling

In [ ]:
%time df = bundle(nodes_df, edges_df, initial_bandwidth=0.05, decay=1.0) # Decay from 0.7 to 1.0

In [ ]:
render_lines(df, width=2000, height=2000, cmap=inferno, bgcolor='black')

#### Decrease initial bandwidth for edge bundling

In [ ]:
%time df = bundle(nodes_df, edges_df, initial_bandwidth=0.025, decay=0.7) # Initial bandwidth from 0.05 to 0.025

In [ ]:
render_lines(df, width=2000, height=2000, cmap=inferno, bgcolor='black')

#### Increase initial bandwidth for edge bundling

In [ ]:
%time df = bundle(nodes_df, edges_df, initial_bandwidth=0.1, decay=0.7) # Initial bandwidth from 0.05 to 0.1

In [ ]:
render_lines(df, width=2000, height=2000, cmap=inferno, bgcolor='black')